In [14]:
!pip install yargy
!pip install wikipedia

In [0]:
import wikipedia
wikipedia.set_lang("ru")

from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

import warnings
warnings.filterwarnings('ignore')

START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN'),
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

START_N = rule(
    gram('NOUN'),
)

START_T = rule(
    or_(
        rule(gram('PREP')),
        rule(gram('NOUN'))
    ).optional()
)

KAK = eq('как')

INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),
)

INCLUDING_S = or_(
    eq('называют'),
    eq('называется'),
    eq('называть')
)


MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
    )

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или')
        ),
        eq('другие')
    ),
    rule(
        eq('—'),
        eq('то'),
        eq('есть'),
        eq(',')
    )
)

MID_N = or_(
    rule(
        eq('есть')
    ),
    rule(
        eq('есть'),
        eq('и'),
        eq('другое'),
        eq('значение'),
        eq('-'),
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('PREP'),
        gram('ADJF'),
        gram('NPRO')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('NPRO'),
    ),
    rule(
        gram('NOUN'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('ADJF'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('CONJ').repeatable(),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
        gram('VERB'),
        gram('ADVB'),
        gram('ADJF')
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')        
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('ADVB'),
        gram('PRTF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()        
    ),
    rule(
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('VERB'),
        gram('NOUN'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        eq('-'),
        gram('CONJ'),
        gram('VERB'),
        gram('ADJF'),
        gram('NOUN').repeatable(),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('VERB'),
        gram('ADJF'),
        gram('NOUN'),
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('VERB'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN'),
        gram('PREP'),
        gram('NPRO'),
        gram('NOUN')
    ),
    rule(
        gram('ADVB'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF')
    ),
    rule(
        gram('PRCL'),
        gram('PRTF'),
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADVB'),
        gram('NOUN')
    ),
    rule(
        gram('ADVB'),
        gram('ADJF')
    ),
    rule(
        gram('CONJ'),
        gram('ADVB'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('NPRO'),
        gram('NOUN').repeatable(),
    ),
    rule(
        gram('ADJF'),
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('ADJF').repeatable()
    ),
    rule(
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN'),
        gram('VERB'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
    ),
    rule(
        gram('ADJF'),
        gram('CONJ'),
        gram('NOUN'),
        gram('CONJ'),
        gram('ADJF'),
        gram('ADJF').repeatable(),
    ),
    rule(
        gram('ADJF'),
        gram('INFN'),
        gram('CONJ'),
        gram('INFN'),
    ),
    rule(
        gram('CONJ'),
        gram('NOUN'),
        gram('VERB'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('VERB'),
        gram('ADVB'),
        gram('ADJF'),
        gram('NOUN'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
    ),
    rule(
        gram('PREP'),
        gram('ADJF'),
        gram('VERB'),
        eq('16'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('VERB'),
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('VERB'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('VERB'),
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADVB'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJS'),
        eq('-'),
        gram('ADJF'),
        gram('ADJS'),
        eq('-'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJS'),
        eq('-'),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN'),
        gram('ADJF'),
        eq('-'),
        eq('л'),
        eq('.'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        eq('-'),
        gram('CONJ'),
        gram('NOUN'),
        gram('CONJ'),
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('CONJ'),
        gram('ADVB'),
        gram('VERB'),
        gram('ADJF'),
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN')
    ),
    rule(
        gram('PRTS')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN'),
        gram('NPRO'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable(),
        gram('ADVB'),
        gram('VERB'),
        gram('PREP'),
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('ADJF'),
        gram('ADVB'),
        gram('VERB')
    ),
    rule(
        gram('ADVB'),
        gram('PRTF'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN')
    ),
    rule(
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJS'),
        eq('-'),
        gram('ADJF'),
        gram('NOUN')
    ),
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [16]:
pip install google

In [0]:
import requests

In [0]:
from bs4 import BeautifulSoup
try: 
  from googlesearch import search 
except ImportError:  
  print("No module named 'google' found") 
def get_hyperonyms(main_word):
  for i in search(main_word + ' это',        # The query you want to run
                tld = 'com',  # The top level domain
                lang = 'en',  # The language
                num = 10,     # Number of results per page
                start = 0,    # First result to retrieve
                stop = 10,  # Last result to retrieve
                pause = 2.0,  # Lapse between HTTP requests
               ):
    res = requests.get(i)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    text = soup.find_all(text=True)

    output = ''
    blacklist = [
      '[document]',
      'noscript',
      'header',
      'html',
      'meta',
      'head', 
      'input',
      'script',
      'style'
    ]

    for t in text:
      if t.parent.name not in blacklist:
        output += '{} '.format(t)

    HYPONYM = eq(utils.deaccent(main_word))
    text = output
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(HYPONYM, MID, END, eq(','), END, eq(','), END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM),
        rule(START, MID, END),
        rule(START_N, INCLUDING_S, END),
        rule(HYPONYM, MID, eq('.'), END),
        rule(START_N, MID, END),
        rule(HYPONYM, MID, END, eq(','), END),
        rule(START_T, HYPONYM, MID_N, END),
        rule(HYPONYM, START, MID, END),
        rule(HYPONYM, MID, END, eq(','), END, eq(','), END),
        rule(END, eq(','), END, eq(','), INCLUDING_S, START_N),
        rule(END, END, START_N, KAK, END, eq(','), END),
        rule(END, START_N, KAK, END),
        rule(END, START_N, INCLUDING_S, END),
        rule(HYPONYM, eq(','), END, eq(','), END, eq(';'), END, eq(','), END)
    )
    parser = Parser(RULE)
    text = utils.deaccent(text)
    text = re.sub(r'\(.+?\)', '', text)
    hyps = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hyps.append(k)
    print(main_word, hyps)
    return hyps

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
import pandas as pd

CHARGE_NUMBER = 18

df = pd.read_csv("/content/gdrive/My Drive/lab2_words.csv")
charge = df.loc[df['IN CHARGE NUMBER'] == CHARGE_NUMBER]
charge.head(3)

,Заполнено,Unnamed: 1,IN CHARGE NUMBER,IN CHARGE NAME,HYPONYM,HYPERNYM,HYPERNYM.1,HYPERNYM.2,HYPERNYM.3,HYPERNYM.4,HYPERNYM.5,HYPERNYM.6,HYPERNYM.7,HYPERNYM.8,HYPERNYM.9,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,#ERROR!
408,NaN,409,18,Bekarys Kuan,ЗВЕРОВОДСТВО,ЖИВОТНОВОДСТВА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
409,NaN,410,18,Bekarys Kuan,ЗДРАВИЦА,ТОСТ,РЕЧЬ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,NaN,411,18,Bekarys Kuan,ЗЕМЛЕРОЙКА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
for index, row in charge.iterrows():
    print(row['HYPONYM'])
    get_hyperonyms(row['HYPONYM'])

ЗВЕРОВОДСТВО
ЗВЕРОВОДСТВО []
ЗДРАВИЦА
ЗДРАВИЦА [['здравица', '—', 'википедия'], ['из', 'википедии', '—', 'свободнои', 'энциклопедии'], ['здравица', '—', 'древнерусскии', 'обряд'], ['современном', 'языке', '—', 'обиходныи', 'термин'], ['нестор', '—', 'история'], ['нестор', '—', 'история']]
ЗЕМЛЕРОЙКА
ЗЕМЛЕРОЙКА []
ЗЕМСНАРЯД
ЗЕМСНАРЯД [['земснаряд', '—', 'википедия'], ['из', 'википедии', '—', 'свободнои', 'энциклопедии'], ['земснаряд', '—', 'судно', 'технического', 'флота'], ['землесосныи', 'снаряд', '—', 'извлекающие', 'и', 'перекачивающие', 'грунт', 'в', 'виде', 'пульпы', 'с', 'применением'], ['скалодробильные', '—', 'предназначены'], ['рефулерныи', '—', 'транспортировка', 'грунта'], ['шаландовыи', '—', 'транспортировка', 'грунта'], ['шаландами', '—', 'специальными', 'судами'], ['самоотвозныи', '—', 'грунт'], ['лонгкулуарные', '—', 'грунт'], ['траншеиным', 'способом', '—', 'земснаряд'], ['папильонажным', 'способом', '—', 'земснаряд']]
ЗЕРНОВКА
ЗЕРНОВКА []
ЗИМОВЩИК
ЗИМОВЩИК [['зимовщик'